In [2]:
import psycopg2
import pandas as pd

In [3]:
conn = psycopg2.connect("dbname=mimic user=postgres password=postgres")
cur = conn.cursor()
sql = '''
SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema';
'''
pd.read_sql_query(sql, conn)

/opt/anaconda3/envs/bd4h_project/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,mimiciii,admissions,postgres,None,True,False,False,False
1,mimiciii,callout,postgres,None,True,False,False,False
2,mimiciii,caregivers,postgres,None,True,False,False,False
3,mimiciii,chartevents_1,postgres,None,True,False,False,False
4,mimiciii,chartevents_2,postgres,None,True,False,False,False
5,mimiciii,chartevents_3,postgres,None,True,False,False,False
6,mimiciii,chartevents_4,postgres,None,True,False,False,False
7,mimiciii,chartevents_5,postgres,None,True,False,False,False
8,mimiciii,chartevents_6,postgres,None,True,False,False,False
9,mimiciii,chartevents_7,postgres,None,True,False,False,False


In [4]:
sql = '''
SELECT COUNT(*)
FROM mimiciii.admissions;
'''
pd.read_sql_query(sql, conn)

/opt/anaconda3/envs/bd4h_project/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,58976


In [44]:
sql = '''
SELECT adm.subject_id, adm.hadm_id, short_title
FROM mimiciii.admissions adm
LEFT JOIN mimiciii.diagnoses_icd diag
ON (adm.subject_id = diag.subject_id AND adm.hadm_id = diag.hadm_id)
LEFT JOIN mimiciii.d_icd_diagnoses icd_diag
ON diag.icd9_code = icd_diag.icd9_code;
'''
df = pd.read_sql_query(sql, conn)

/opt/anaconda3/envs/bd4h_project/lib/python3.10/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [45]:
df['short_title'] = df['short_title'].astype(str)

In [46]:
df['symptoms'] = df.groupby(['subject_id', 'hadm_id']).transform(lambda x: ','.join(x))
df_final = df.drop_duplicates(subset=['subject_id', 'hadm_id'])

In [47]:
df_final['hadm_id'] = df_final['hadm_id'].astype(str)
df_final['input'] = df_final[['hadm_id', 'symptoms']].agg('_'.join, axis=1)
df_final

/var/folders/ns/vlsp2f8j13s4b4_v6_r3yzz40000gn/T/ipykernel_38684/3868676960.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['hadm_id'] = df_final['hadm_id'].astype(str)
/var/folders/ns/vlsp2f8j13s4b4_v6_r3yzz40000gn/T/ipykernel_38684/3868676960.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['input'] = df_final[['hadm_id', 'symptoms']].agg('_'.join, axis=1)


,subject_id,hadm_id,short_title,symptoms,input
0,109,172335,Mal hyp kid w cr kid V,"Mal hyp kid w cr kid V,Pneumonia, organism NOS...","172335_Mal hyp kid w cr kid V,Pneumonia, organ..."
14,109,173633,Mal hyp kid w cr kid V,"Mal hyp kid w cr kid V,End stage renal disease...","173633_Mal hyp kid w cr kid V,End stage renal ..."
28,112,174105,Ac stomach ulcer w hem,"Ac stomach ulcer w hem,Subendo infarct, initia...","174105_Ac stomach ulcer w hem,Subendo infarct,..."
33,113,109976,Mal neo cereb ventricle,"Mal neo cereb ventricle,Obstructiv hydrocephal...","109976_Mal neo cereb ventricle,Obstructiv hydr..."
36,114,178393,Crnry athrscl natve vssl,"Crnry athrscl natve vssl,Intermed coronary syn...","178393_Crnry athrscl natve vssl,Intermed coron..."
...,...,...,...,...,...
650983,97484,172304,Oth lymp unsp xtrndl org,"Oth lymp unsp xtrndl org,Cerebral edema,Aphasi...","172304_Oth lymp unsp xtrndl org,Cerebral edema..."
650989,97488,152542,Anal & rectal abscess,"Anal & rectal abscess,DMII neuro uncntrld,Neur...","152542_Anal & rectal abscess,DMII neuro uncntr..."
651000,97488,161999,Crbl emblsm w infrct,"Crbl emblsm w infrct,Cerebral edema,Compressio...","161999_Crbl emblsm w infrct,Cerebral edema,Com..."
651024,97492,189314,Othr migrne wo ntrc mgrn,"Othr migrne wo ntrc mgrn,Aphasia,Secundum atri...","189314_Othr migrne wo ntrc mgrn,Aphasia,Secund..."


In [57]:
import csv
df_final['input'].to_csv('input.txt', header=None, index=None,quoting = csv.QUOTE_NONE, escapechar = ' ', sep=' ')
